# Setup
Loading Dataset and getting basic general info

In [1]:
from datasets import load_dataset
ds = load_dataset('multi_news')

import pandas as pd
df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['test'])
import sys
sys.path.append(r'notebooks/python_scripts')
from LLMscripts import *

b:\LHL\LHL-LLM\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
b:\LHL\LHL-LLM\.conda\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ElenaFall\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_train.head()

,document,summary
0,"National Archives \n \n Yes, it’s that time ag...",– The unemployment rate dropped to 8.2% last m...
1,LOS ANGELES (AP) — In her first interview sinc...,"– Shelly Sterling plans ""eventually"" to divorc..."
2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...
3,Tucker Carlson Exposes His Own Sexism on Twitt...,– Tucker Carlson is in deep doodoo with conser...
4,A man accused of removing another man's testic...,– What are the three most horrifying words in ...


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44972 entries, 0 to 44971
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  44972 non-null  object
 1   summary   44972 non-null  object
dtypes: object(2)
memory usage: 702.8+ KB


In [4]:
df_train.describe()

,document,summary
count,44972,44972
unique,44911,44972
top,Tweet with a location \n \n You can add locati...,– The unemployment rate dropped to 8.2% last m...
freq,11,1


In [5]:
df_train.isnull().sum()

document    0
summary     0
dtype: int64

# Preprossesing
- Removing punctuation
- Remove stop words

## Punctuation

In [6]:
df_train['document'] = df_train['document'].apply(remove_punctuation)
df_test['document'] = df_test['document'].apply(remove_punctuation)

## Stop Words

In [ ]:
df_train['document'] = df_train['document'].apply(remove_stopwords)
df_test['document'] = df_test['document'].apply(remove_stopwords)

In [ ]:
import pickle

with open('pickles/df_train_prepro.pickle', 'wb') as f:
    pickle.dump(df_train, f)

with open('pickles/df_test_prepro.pickle', 'wb') as f:
    pickle.dump(df_test, f)

